<a href="https://colab.research.google.com/github/ysnkchk0727-rgb/-/blob/main/cardevcont5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 🔹 멀티모달 LLM 계열인 CLIP 사용을 위한 라이브러리 설치
!pip install open_clip_torch --quiet
!pip install pandas --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile
import os

# 🔹 Google Drive에 있는 압축 파일 경로 (네가 사용하던 경로 그대로)
zip_path = "/content/drive/MyDrive/train_data.zip"

# 🔹 압축을 풀 위치
extract_path = "/content/train_data"

# 이미 풀려 있으면 다시 안 풀어도 됨
if not os.path.exists(extract_path) or len(os.listdir(extract_path)) == 0:
    os.makedirs(extract_path, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(extract_path)
    print("압축 해제 완료:", extract_path)
else:
    print("이미 압축이 풀려 있습니다:", extract_path)

Mounted at /content/drive
압축 해제 완료: /content/train_data


In [3]:
import os
import random
import torch
from torchvision import datasets
from torch.utils.data import Subset, DataLoader

import open_clip  # 방금 설치한 라이브러리

# 🔹 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print("사용 디바이스:", device)

# 🔹 CLIP 모델(비전-언어 멀티모달 LLM) 불러오기
# ViT-B-32 버전: 속도/메모리 적당한 편
model, _, preprocess = open_clip.create_model_and_transforms(
    'ViT-B-32',
    pretrained='openai'
)
tokenizer = open_clip.get_tokenizer('ViT-B-32')
model.to(device)
model.eval()

# 🔹 ImageFolder로 데이터셋 로드 (폴더명이 라벨)
data_dir = "/content/train_data"
full_dataset = datasets.ImageFolder(data_dir, transform=preprocess)

print("클래스(폴더) 이름:", full_dataset.classes)
num_classes = len(full_dataset.classes)
print("클래스 개수:", num_classes)
print("총 이미지 개수:", len(full_dataset))

# 🔹 한 번만 쓸 "고정된 train/val 분할" 만들기
num_samples = len(full_dataset)
indices = list(range(num_samples))

# seed 고정 → 항상 같은 방식으로 섞임
random.seed(42)
random.shuffle(indices)

train_ratio = 0.8
train_size = int(num_samples * train_ratio)
train_indices = indices[:train_size]
val_indices = indices[train_size:]

print("train 개수:", len(train_indices))
print("val 개수:", len(val_indices))

# 🔹 Subset으로 train/val 나누기
train_dataset = Subset(full_dataset, train_indices)
val_dataset = Subset(full_dataset, val_indices)

# DataLoader (CLIP zero-shot이니까 shuffle은 굳이 필요 없음)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

사용 디바이스: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


클래스(폴더) 이름: ['front', 'front_left', 'front_right', 'inside', 'left', 'rear', 'rear_left', 'rear_right', 'right', 'tire']
클래스 개수: 10
총 이미지 개수: 100
train 개수: 80
val 개수: 20


In [4]:
# 🔹 폴더 이름(라벨) 순서
class_names = full_dataset.classes
print("class_names:", class_names)

# 🔹 각 클래스에 대한 텍스트 설명(프롬프트) 정의
prompt_templates = {
    "front":       "a photo of the front of a car",
    "rear":        "a photo of the rear of a car",
    "left":        "a photo of the left side of a car",
    "right":       "a photo of the right side of a car",
    "front_left":  "a front-left diagonal view of a car",
    "front_right": "a front-right diagonal view of a car",
    "rear_left":   "a rear-left diagonal view of a car",
    "rear_right":  "a rear-right diagonal view of a car",
    "inside":      "an interior photo of a car showing seats or dashboard",
    "tire":        "a close-up photo of a car tire or wheel"
}

# 🔹 ImageFolder가 가진 class 순서에 맞춰서 문장 배열 만들기
texts = [prompt_templates[c] for c in class_names]
print("사용할 텍스트 프롬프트:")
for t in texts:
    print(" -", t)

# 🔹 텍스트를 CLIP text encoder로 임베딩
text_tokens = tokenizer(texts).to(device)

with torch.no_grad():
    text_features = model.encode_text(text_tokens)
    text_features /= text_features.norm(dim=-1, keepdim=True)  # 정규화

class_names: ['front', 'front_left', 'front_right', 'inside', 'left', 'rear', 'rear_left', 'rear_right', 'right', 'tire']
사용할 텍스트 프롬프트:
 - a photo of the front of a car
 - a front-left diagonal view of a car
 - a front-right diagonal view of a car
 - an interior photo of a car showing seats or dashboard
 - a photo of the left side of a car
 - a photo of the rear of a car
 - a rear-left diagonal view of a car
 - a rear-right diagonal view of a car
 - a photo of the right side of a car
 - a close-up photo of a car tire or wheel


In [5]:
from tqdm import tqdm

def evaluate_clip_zero_shot(model, data_loader, text_features, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(data_loader):
            images = images.to(device)
            labels = labels.to(device)

            # 이미지 임베딩
            image_features = model.encode_image(images)
            image_features /= image_features.norm(dim=-1, keepdim=True)

            # 이미지 vs 텍스트 유사도 (내적)
            logits = 100.0 * image_features @ text_features.T  # [batch, 10]
            preds = torch.argmax(logits, dim=-1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = correct / total * 100.0
    return acc

# 🔹 train/val 정확도 계산
train_acc = evaluate_clip_zero_shot(model, train_loader, text_features, device)
val_acc = evaluate_clip_zero_shot(model, val_loader, text_features, device)

print(f"CLIP zero-shot Train Accuracy: {train_acc:.2f}%")
print(f"CLIP zero-shot Val   Accuracy: {val_acc:.2f}%")

100%|██████████| 2/2 [00:03<00:00,  1.60s/it]

CLIP zero-shot Train Accuracy: 40.00%
CLIP zero-shot Val   Accuracy: 30.00%


In [6]:
from PIL import Image
import pandas as pd

rows = []

# 🔹 full_dataset.samples: (이미지 경로, 정답 라벨 인덱스) 리스트
for img_path, _ in tqdm(full_dataset.samples):
    # 이미지 로드
    img = Image.open(img_path).convert("RGB")

    # CLIP 전처리 적용
    img_tensor = preprocess(img).unsqueeze(0).to(device)  # [1, C, H, W]

    with torch.no_grad():
        img_feat = model.encode_image(img_tensor)
        img_feat /= img_feat.norm(dim=-1, keepdim=True)

        logits = 100.0 * img_feat @ text_features.T
        pred_idx = int(torch.argmax(logits, dim=-1).item())

    pred_label = class_names[pred_idx]
    filename = os.path.basename(img_path)

    rows.append({"filename": filename, "prediction": pred_label})

df = pd.DataFrame(rows)
save_path = "/content/clip_result.csv"
df.to_csv(save_path, index=False)

print("예측 결과 CSV 저장 완료:", save_path)

100%|██████████| 100/100 [00:25<00:00,  3.87it/s]

예측 결과 CSV 저장 완료: /content/clip_result.csv


In [7]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# 혹시 모를 상황 대비해서 다시 한 번 확인
print("train 개수:", len(train_dataset))
print("val 개수:", len(val_dataset))

def extract_features(dataset, model, device):
    """
    dataset: Subset(full_dataset, indices)
    model: CLIP 모델 (image encoder 사용)
    return: (features, labels)
    """
    loader = DataLoader(dataset, batch_size=16, shuffle=False)
    all_feats = []
    all_labels = []

    model.eval()
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Extracting features"):
            imgs = imgs.to(device)
            # CLIP 이미지 임베딩
            feats = model.encode_image(imgs)
            # 정규화 (cosine similarity 기반이라 보통 이렇게 맞춰줌)
            feats = feats / feats.norm(dim=-1, keepdim=True)

            all_feats.append(feats.cpu())
            all_labels.append(labels)

    all_feats = torch.cat(all_feats, dim=0)   # [N, D]
    all_labels = torch.cat(all_labels, dim=0) # [N]
    return all_feats, all_labels

train_feats, train_labels = extract_features(train_dataset, model, device)
val_feats, val_labels = extract_features(val_dataset, model, device)

print("train_feats shape:", train_feats.shape)
print("val_feats shape:", val_feats.shape)

train 개수: 80
val 개수: 20


Extracting features: 100%|██████████| 2/2 [00:03<00:00,  1.61s/it]

train_feats shape: torch.Size([80, 512])
val_feats shape: torch.Size([20, 512])


In [8]:
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# 🔹 feature 차원 & 클래스 수
feature_dim = train_feats.shape[1]
num_classes = len(full_dataset.classes)
print("feature_dim:", feature_dim, "num_classes:", num_classes)

# 🔹 간단한 Linear 분류기 (softmax 분류)
classifier = nn.Linear(feature_dim, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-2, weight_decay=1e-4)

# 🔹 feature/label로 구성된 Dataset & DataLoader
train_ds_feat = TensorDataset(train_feats, train_labels)
val_ds_feat = TensorDataset(val_feats, val_labels)

train_loader_feat = DataLoader(train_ds_feat, batch_size=16, shuffle=True)
val_loader_feat = DataLoader(val_ds_feat, batch_size=16, shuffle=False)

def train_classifier_one_epoch(classifier, loader, optimizer, criterion, device):
    classifier.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for feats, labels in loader:
        feats = feats.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = classifier(feats)  # [B, num_classes]
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * feats.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += feats.size(0)

    return total_loss / total, correct / total * 100.0

def eval_classifier(classifier, loader, criterion, device):
    classifier.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for feats, labels in loader:
            feats = feats.to(device)
            labels = labels.to(device)

            outputs = classifier(feats)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * feats.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += feats.size(0)

    return total_loss / total, correct / total * 100.0

EPOCHS = 50
best_val_acc = 0.0

for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = train_classifier_one_epoch(
        classifier, train_loader_feat, optimizer, criterion, device
    )
    val_loss, val_acc = eval_classifier(
        classifier, val_loader_feat, criterion, device
    )

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_state = classifier.state_dict()

    print(f"[Epoch {epoch:02d}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

print("\n최고 검증 정확도 (CLIP feature + Linear classifier): "
      f"{best_val_acc:.2f}%")

# 🔹 최고 성능일 때의 가중치로 되돌리기
classifier.load_state_dict(best_state)

feature_dim: 512 num_classes: 10
[Epoch 01] Train Loss: 2.2993, Train Acc: 12.50% | Val Loss: 2.2766, Val Acc: 15.00%
[Epoch 02] Train Loss: 2.1509, Train Acc: 47.50% | Val Loss: 2.2362, Val Acc: 30.00%
[Epoch 03] Train Loss: 2.0421, Train Acc: 56.25% | Val Loss: 2.2092, Val Acc: 30.00%
[Epoch 04] Train Loss: 1.9286, Train Acc: 62.50% | Val Loss: 2.1598, Val Acc: 30.00%
[Epoch 05] Train Loss: 1.8305, Train Acc: 70.00% | Val Loss: 2.1027, Val Acc: 35.00%
[Epoch 06] Train Loss: 1.7390, Train Acc: 72.50% | Val Loss: 2.0418, Val Acc: 40.00%
[Epoch 07] Train Loss: 1.6483, Train Acc: 80.00% | Val Loss: 1.9810, Val Acc: 40.00%
[Epoch 08] Train Loss: 1.5602, Train Acc: 81.25% | Val Loss: 1.9314, Val Acc: 40.00%
[Epoch 09] Train Loss: 1.4789, Train Acc: 83.75% | Val Loss: 1.8933, Val Acc: 40.00%
[Epoch 10] Train Loss: 1.4067, Train Acc: 86.25% | Val Loss: 1.8519, Val Acc: 40.00%
[Epoch 11] Train Loss: 1.3369, Train Acc: 87.50% | Val Loss: 1.8131, Val Acc: 40.00%
[Epoch 12] Train Loss: 1.2744, T

<All keys matched successfully>

In [9]:
from PIL import Image
import pandas as pd

rows = []

# 🔹 full_dataset 순서대로 feature 뽑고 → classifier 통과
full_loader = DataLoader(full_dataset, batch_size=16, shuffle=False)

model.eval()
classifier.eval()

with torch.no_grad():
    for imgs, labels in tqdm(full_loader, desc="Predicting all images"):
        imgs = imgs.to(device)
        # CLIP 이미지 feature
        feats = model.encode_image(imgs)
        feats = feats / feats.norm(dim=-1, keepdim=True)

        logits = classifier(feats)  # [B, num_classes]
        preds = logits.argmax(dim=1).cpu().tolist()

        # batch 안의 각 이미지에 대해 filename + pred_label 저장
        # DataLoader는 dataset.__getitem__ 순서를 그대로 사용하므로,
        # index를 따로 관리할 필요 없이 dataset.samples에서 경로를 꺼낼 수 있어.
        # 다만, 여기서는 조금 쉽게 하기 위해, 반복문 외부에서 전체 파일 리스트를 같이 돌리자.
        # → 아래에서 별도로 처리할게요.

Predicting all images: 100%|██████████| 7/7 [00:18<00:00,  2.69s/it]


In [10]:
rows = []
model.eval()
classifier.eval()

with torch.no_grad():
    for idx in tqdm(range(len(full_dataset)), desc="Predicting"):
        img, _ = full_dataset[idx]
        img = img.to(device).unsqueeze(0)  # [1, C, H, W]

        feat = model.encode_image(img)
        feat = feat / feat.norm(dim=-1, keepdim=True)

        logits = classifier(feat)  # [1, num_classes]
        pred_idx = int(logits.argmax(dim=1).item())
        pred_label = full_dataset.classes[pred_idx]

        # 파일 경로/이름
        img_path, _ = full_dataset.samples[idx]
        filename = os.path.basename(img_path)

        rows.append({"filename": filename, "prediction": pred_label})

df = pd.DataFrame(rows)
save_path = "/content/clip_linear_result.csv"
df.to_csv(save_path, index=False)
print("예측 결과 CSV 저장 완료:", save_path)

Predicting: 100%|██████████| 100/100 [00:26<00:00,  3.75it/s]

예측 결과 CSV 저장 완료: /content/clip_linear_result.csv


In [11]:
import numpy as np
from collections import Counter
from tqdm import tqdm

# val_dataset: Subset(full_dataset, val_indices)
# classifier, model, device, full_dataset.classes 가 이미 정의되어 있다고 가정

num_classes = len(full_dataset.classes)
class_names = full_dataset.classes

# 전체 confusion matrix와 클래스별 카운트
conf_mat = np.zeros((num_classes, num_classes), dtype=int)
val_counts = Counter()
val_correct_counts = Counter()

model.eval()
classifier.eval()

for i in tqdm(range(len(val_dataset)), desc="Val 분석"):
    img, true_label = val_dataset[i]  # img: tensor, true_label: int
    img = img.to(device).unsqueeze(0) # [1, C, H, W]

    with torch.no_grad():
        feat = model.encode_image(img)
        feat = feat / feat.norm(dim=-1, keepdim=True)
        logits = classifier(feat)
        pred_label = int(logits.argmax(dim=1).item())

    conf_mat[true_label, pred_label] += 1
    val_counts[true_label] += 1
    if pred_label == true_label:
        val_correct_counts[true_label] += 1

print("\n[클래스별 검증 정확도]")
for cls_idx, cls_name in enumerate(class_names):
    total = val_counts[cls_idx]
    correct = val_correct_counts[cls_idx]
    acc = (correct / total * 100) if total > 0 else 0.0
    print(f"{cls_name:12s} : {correct:2d} / {total:2d}  ({acc:.1f}%)")

print("\n[Confusion Matrix] (행=실제, 열=예측)")
print("     " + "  ".join([f"{name[:4]:>4s}" for name in class_names]))
for i, row in enumerate(conf_mat):
    row_str = "  ".join([f"{v:4d}" for v in row])
    print(f"{class_names[i][:4]:>4s}  {row_str}")

Val 분석: 100%|██████████| 20/20 [00:07<00:00,  2.78it/s]


[클래스별 검증 정확도]
front        :  2 /  2  (100.0%)
front_left   :  0 /  4  (0.0%)
front_right  :  1 /  3  (33.3%)
inside       :  2 /  2  (100.0%)
left         :  0 /  0  (0.0%)
rear         :  1 /  1  (100.0%)
rear_left    :  1 /  2  (50.0%)
rear_right   :  1 /  1  (100.0%)
right        :  1 /  3  (33.3%)
tire         :  2 /  2  (100.0%)

[Confusion Matrix] (행=실제, 열=예측)
     fron  fron  fron  insi  left  rear  rear  rear  righ  tire
fron     2     0     0     0     0     0     0     0     0     0
fron     0     0     1     0     0     0     0     2     1     0
fron     1     0     1     0     0     0     0     1     0     0
insi     0     0     0     2     0     0     0     0     0     0
left     0     0     0     0     0     0     0     0     0     0
rear     0     0     0     0     0     1     0     0     0     0
rear     0     0     0     0     0     0     1     1     0     0
rear     0     0     0     0     0     0     0     1     0     0
righ     0     0     0     0     2     0     

In [12]:
import torch
import torch.nn as nn
from torch.utils.data import Subset, DataLoader
from torchvision.models import vit_b_16, ViT_B_16_Weights
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
print("사용 디바이스:", device)

# 1) 같은 split 사용: train_indices, val_indices
weights = ViT_B_16_Weights.IMAGENET1K_V1
vit_transform = weights.transforms()  # ViT 권장 전처리

vit_full_dataset = datasets.ImageFolder(data_dir, transform=vit_transform)

vit_train_dataset = Subset(vit_full_dataset, train_indices)
vit_val_dataset   = Subset(vit_full_dataset, val_indices)

vit_train_loader = DataLoader(vit_train_dataset, batch_size=16, shuffle=True)
vit_val_loader   = DataLoader(vit_val_dataset, batch_size=16, shuffle=False)

num_classes = len(vit_full_dataset.classes)
print("클래스 수:", num_classes, "→", vit_full_dataset.classes)

# 2) ViT 모델 정의
vit_model = vit_b_16(weights=weights)
vit_model.heads.head = nn.Linear(vit_model.heads.head.in_features, num_classes)
vit_model = vit_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(vit_model.parameters(), lr=1e-4, weight_decay=1e-3)

def train_vit_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for imgs, labels in tqdm(loader, desc="ViT Train"):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += imgs.size(0)

    return total_loss / total, correct / total * 100.0

def eval_vit(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="ViT Val"):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += imgs.size(0)

    return total_loss / total, correct / total * 100.0

EPOCHS = 10
best_vit_val_acc = 0.0
best_vit_state = None

for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = train_vit_one_epoch(vit_model, vit_train_loader, optimizer, criterion, device)
    val_loss, val_acc = eval_vit(vit_model, vit_val_loader, criterion, device)

    print(f"[ViT Epoch {epoch:02d}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

    if val_acc > best_vit_val_acc:
        best_vit_val_acc = val_acc
        best_vit_state = vit_model.state_dict()

print("\nViT 최고 검증 정확도:", best_vit_val_acc, "%")

if best_vit_state is not None:
    vit_model.load_state_dict(best_vit_state)
    print("<Best ViT weights loaded>")

사용 디바이스: cpu
클래스 수: 10 → ['front', 'front_left', 'front_right', 'inside', 'left', 'rear', 'rear_left', 'rear_right', 'right', 'tire']
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth


100%|██████████| 330M/330M [00:06<00:00, 51.8MB/s]
ViT Val: 100%|██████████| 2/2 [00:12<00:00,  6.30s/it]


[ViT Epoch 01] Train Loss: 2.2022, Train Acc: 18.75% | Val Loss: 1.9705, Val Acc: 30.00%


ViT Val: 100%|██████████| 2/2 [00:12<00:00,  6.29s/it]


[ViT Epoch 02] Train Loss: 0.9988, Train Acc: 82.50% | Val Loss: 1.1904, Val Acc: 55.00%


...

[ViT Epoch 08] Train Loss: 0.0071, Train Acc: 100.00% | Val Loss: 0.8634, Val Acc: 70.00%


ViT Val: 100%|██████████| 2/2 [00:12<00:00,  6.17s/it]


[ViT Epoch 09] Train Loss: 0.0053, Train Acc: 100.00% | Val Loss: 0.8964, Val Acc: 70.00%


ViT Val: 100%|██████████| 2/2 [00:12<00:00,  6.03s/it]

[ViT Epoch 10] Train Loss: 0.0043, Train Acc: 100.00% | Val Loss: 0.9207, Val Acc: 65.00%

ViT 최고 검증 정확도: 70.0 %
<Best ViT weights loaded>
